In [ ]:
import rosbag
import rospy

import yaml
from rosbag.bag import Bag
p = "/home/jonfrey/Datasets/labdata/2021-06-17-17-56-33.bag"
with rosbag.Bag(p, 'r') as bag:
    pass
info_dict = yaml.load(Bag(p, 'r')._get_yaml_info())
info_dict

In [ ]:
from sensor_msgs.msg import CameraInfo as RosCameraInfo
camera_info = RosCameraInfo()
camera_inf = RosCameraInfo()
camera_inf.header.frame_id ="2"
camera_info.header = camera_inf.header
camera_info

In [ ]:
import geometry_msgs  
import sensor_msgs
from sensor_msgs.msg import Image as RosImage
from geometry_msgs.msg import Pose as RosPose
import rosbag
import rospy

import cv2
from sensor_msgs.msg import CameraInfo as RosCameraInfo
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
from PIL import Image
import copy 
class KimeraAdapterNode():
    def __init__(self):
        rospy.init_node('kimera_adapter_node')
        self.bridge = CvBridge()
        self.pub_img = rospy.Publisher('/mono/image_raw', RosImage, queue_size=10)
        self.pub_camera_info = rospy.Publisher('/mono/camera_info', RosImage, queue_size=10)
        self.pub_pose = rospy.Publisher('/vicon/firefly_sbx/firefly_sbx', RosPose, queue_size=10)

        rospy.Subscriber("/rgb/image_raw", RosImage, self.callback_rgb)
        rospy.Subscriber("/rgb/camera_info", RosCameraInfo, self.callback_rgb_camera_info)
        
        self.map1, self.map2 = None, None
        self.camera_info = None
        
        self.po = RosPose()
        self.po.orientation.w = 1
        self.r = rospy.Rate(10)
        
    def run(self):
        print("Running")
        self.pub_pose.publish(self.po)
        self.r.sleep()

    
    def callback_rgb(self,data):
        rgb = self.bridge.imgmsg_to_cv2(data)
        if not self.map1 is None:
            rgb_undistorted = cv2.remap(rgb, self.map1, self.map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
            mono = cv2.cvtColor(rgb_undistorted, cv2.COLOR_BGR2GRAY)
            mono_msg = self.bridge.cv2_to_imgmsg(mono)
            display(Image.fromarray(mono))
            mono_msg.header = data.header
            self.pub_img.publish( mono_msg )
        else:
            print("Map not initalized")
    
    def callback_rgb_camera_info(self, data):
        
        if self.camera_info is None:
            self.camera_info = data
            h,  w = data.height,data.width
            K = np.array( data.K ).reshape(3,3)
            D = np.array( data.D )
            
            new_K, validPixROI = cv2.getOptimalNewCameraMatrix( K, D[None,:],(w,h), alpha = 0)
            self.map1, self.map2 = cv2.initUndistortRectifyMap( K , D[None,:], np.eye(3), new_K, (w,h) , cv2.CV_16SC2)
            
            self.camera_info.K = new_K.reshape(-1)
            self.camera_info.D = D.fill(0)
            self.camera_info.header = data.header
            print("type(data)1", type(data), type(self.camera_info))
        
            self.pub_camera_info.publish(copy.deepcopy( self.camera_info) ) 
            return
        self.camera_info.header = data.header
        print("type(data)", type(data), type(self.camera_info))
        
        self.pub_camera_info.publish(copy.deepcopy( self.camera_info) ) 
        
       
        
kan = KimeraAdapterNode()

print("STARTED", rospy.is_shutdown())
while not rospy.is_shutdown():
    kan.run()
    print("Run")

In [ ]:
from sensor_msgs.msg import CameraInfo as RosCameraInfo
camera_info = RosCameraInfo()

#camera_info.height = 
#camera_info.width =
import numpy as np
camera_info.header

In [ ]:


RosImage
dir(bridge)
import numpy as np


In [ ]:
rosimg = RosImage()
dir(rosimg.serialize_numpy())

In [ ]:
result

In [ ]:
from cv_bridge import CvBridge
bridge = CvBridge()

with rosbag.Bag(p, 'r') as bag:
    count = 0
    max_counts = 1
    image_topics = [t['topic'] for t in info_dict['topics'] if t['topic'].find('image') != -1]
    
    out = {}
    for topic, msg, t in bag.read_messages( topics=image_topics ):
        image = bridge.imgmsg_to_cv2(msg , desired_encoding='passthrough')
        try:
            out[topic].append(image)
        except:
            out[topic] = [image]
        count += 1

        if count > len(image_topics)*max_counts: break

In [ ]:
out['/rgb/image_raw'][0].shape

In [ ]:
with rosbag.Bag(p, 'r') as bag:
    for topic, msg, t in bag.read_messages( topics="/points2"):
        print(topic, msg, t)
        break

In [ ]:
import numpy as np
import ros_numpy  # apt install ros-noetic-ros-numpy
import rosbag
import sensor_msgs

def convert_pc_msg_to_np(pc_msg):
    pc_msg.__class__ = sensor_msgs.msg._PointCloud2.PointCloud2
    offset_sorted = {f.offset: f for f in pc_msg.fields}
    pc_msg.fields = [f for (_, f) in sorted(offset_sorted.items())]

    pc_np = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(pc_msg, remove_nans=True)
    return pc_np
    
pcd = convert_pc_msg_to_np( msg )


import matplotlib.pyplot as plt
def plot_birds_eye(pcd,N_max = 1000):
    data = np.random.permutation(pcd)[:1000,:]
    x = data[:,0]
    y = data[:, 1]
    plt.scatter(x, y,s=4, alpha=1)
    
    plt.scatter(0, 0,c="r", alpha=1)
    plt.show()
plot_birds_eye(pcd)

In [ ]:
to = [t['topic'] for t in info_dict['topics'] if t['topic'].find('image') != -1]
to

In [ ]:
from PIL import Image
import numpy as np
white = np.uint8( np.full((1080,50,3), 255 ))
images = [out[k][-1] for k in  list(out.keys())[1:]]
res = [images[0], white, images[1], white, images[2]]

t = np.concatenate(res, axis=1)
Image.fromarray( t )
#     img = Image.fromarray( )
#     display(img)

In [ ]:
if topic == "/tf" and msg.transforms:

In [ ]:
with rosbag.Bag(p, 'r') as bag:
    for topic, msg, t in bag.read_messages( topics="/rgb/camera_info"):
        topic= topic
        msg = msg
        t = t
        print(topic, msg, t)
        break

K = np.array( msg.K ).reshape(3,3)
D = np.array( msg.D )

In [ ]:
listener.allFramesAsDot()

In [ ]:
import tf
# ROS and SCIPY use XYZW
from scipy.spatial.transform import Rotation as R
import rospy
import numpy as np
import rospy

rospy.init_node('as')
listener = tf.TransformListener()

def get(a,b):
    (trans,rot) = listener.lookupTransform(a,b, rospy.Time(0))
    H = np.eye(4)
    H[:3,:3] = R.from_quat(rot).as_matrix() 
    H[:3,3] = trans
    print(trans)
    return H

try:
    H_imu_rgb_lab = get('/pickelhaubeimu_link', "/cam1")
    print(H_imu_rgb_lab)
except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException):
    pass


In [ ]:
from scipy.spatial.transform import Rotation as R

import os
def replace(f, H):
    #input file
    fin = open(f, "rt")
    #output file to write the result to
    fout = open("out.yaml", "wt")
    #for each line in the input file
    for line in fin:
        if line.find("data") != -1:
            print( "  data: " + str(list(H.reshape(-1)) ) +'\n')
            fout.write("  data: " + str(list(H.reshape(-1)) ) +'\n')
        else:
            fout.write(line)
    #close input and output files
    fin.close()
    fout.close()
    os.system( f"cp out.yaml {f}")

    
def replace_K(f, K):
    #input file
    fin = open(f, "rt")
    #output file to write the result to
    fout = open("out.yaml", "wt")
    #for each line in the input file
    for line in fin:
        if line.find("intrinsics") != -1:
            print( "intrinsics: " + str([K[0,0],K[1,1],K[0,2],K[1,2]] ) +'\n')
            fout.write("intrinsics: " + str([K[0,0],K[1,1],K[0,2],K[1,2]]) +'\n')
        else:
            fout.write(line)
    #close input and output files
    fin.close()
    fout.close()
    os.system( f"cp out.yaml {f}")

def replace_gen(f, data, tmp):
    #input file
    fin = open(f, "rt")
    #output file to write the result to
    fout = open("out.yaml", "wt")
    #for each line in the input file
    for line in fin:
        if line.find(tmp) != -1:
            print( f"{tmp}: " + str(data) +'\n')
            fout.write(f"{tmp}: " + str(data) +'\n')
        else:
            fout.write(line)
    #close input and output files
    fin.close()
    fout.close()
    os.system( f"cp out.yaml {f}")

In [ ]:
if use_kinect:
    K = np.load ( "/home/jonfrey/ASL/docker/vio/scripts/K_Kinect.npy")
    a = [1280, 720]
else:
    K = np.load ( "/home/jonfrey/ASL/docker/vio/scripts/K_Cam1.npy")
    a = [1440, 700]
    
replace_gen("/home/jonfrey/ASL/docker/vio/params/labdata/RightCameraParams.yaml",a,"resolution")
replace_gen("/home/jonfrey/ASL/docker/vio/params/labdata/LeftCameraParams.yaml",a,"resolution")
replace_K('/home/jonfrey/ASL/docker/vio/params/labdata/RightCameraParams.yaml', K)
replace_K('/home/jonfrey/ASL/docker/vio/params/labdata/LeftCameraParams.yaml', K)

In [ ]:
print( R.from_quat( [-0.356063, 0.615575 ,0.612224, -0.345642]).as_matrix() )

H_imu_rgb_lab


In [ ]:
l, r

In [ ]:
use_inv = False
# trafo = R.from_euler(['x','z'],[-90,90], degrees=True).as_matrix() 

H_trafo = np.eye(4)
# H_trafo[:3,:3] = trafo
H_trafo_imu = np.eye(4)
replace('/home/jonfrey/ASL/docker/vio/params/labdata/ImuParams.yaml',H_trafo_imu )
l = H_imu_rgb_lab
use_left = False
if use_left:  
    if use_inv:
        replace('/home/jonfrey/ASL/docker/vio/params/labdata/LeftCameraParams.yaml',np.linalg.inv( l @ H_trafo) )
        r = l
        r[1,3] += 0.1
        replace('/home/jonfrey/ASL/docker/vio/params/labdata/RightCameraParams.yaml',np.linalg.inv( r @ H_trafo))
    else:
        replace('/home/jonfrey/ASL/docker/vio/params/labdata/LeftCameraParams.yaml',l @ H_trafo )
        r = l
        r[1,3] += 0.1
        replace('/home/jonfrey/ASL/docker/vio/params/labdata/RightCameraParams.yaml',r @ H_trafo)

        
else:
    if use_inv:
        l[1,3] -= 0.1
        replace('/home/jonfrey/ASL/docker/vio/params/labdata/LeftCameraParams.yaml',np.linalg.inv( l @ H_trafo))
        r = l
        r[1,3] += 0.1
        replace('/home/jonfrey/ASL/docker/vio/params/labdata/RightCameraParams.yaml',np.linalg.inv( r @ H_trafo))
    else:            
        l[1,3] -= 0.1
        replace('/home/jonfrey/ASL/docker/vio/params/labdata/LeftCameraParams.yaml',l @ H_trafo)
        r = l
        r[1,3] += 0.1
        replace('/home/jonfrey/ASL/docker/vio/params/labdata/RightCameraParams.yaml',r @ H_trafo)
    

In [ ]:
new = H_imu_rgb
new[:3,:3] = H_imu_rgb[:3,:3] # @ R.from_euler('zyx', [90, 0, 0], degrees=True).as_matrix()
print( "Left", new.reshape(-1).tolist() ) 

new2 = new
new2[1,3] += 0.1
print("Right", new2.reshape(-1).tolist())

In [ ]:
H_imu_rgb.reshape

In [ ]:
import cv2
import numpy as np
from PIL import Image

img = out['/rgb/image_raw'][0][:,:,:3]
## OpenCV 4.4.0 !!!
display(Image.fromarray(out['/rgb/image_raw'][0][:,:,:3]))
h,  w = img.shape[:2]
retval, validPixROI	= cv2.getOptimalNewCameraMatrix(K ,D[None,:],(w,h), alpha = 0)
map1, map2 = cv2.initUndistortRectifyMap( K , D[None,:], np.eye(3), new_K, (w,h) , cv2.CV_16SC2)

undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT) # cv2.BORDER_CONSTANT cv2.BORDER_REFLECT
display(Image.fromarray(undistorted_img))

new_K

In [ ]:
# PLAN WRITE A ROS-NODE CHOOSE NEW KAMERA MATRIX AT THE START ALWAYS RECTIFY THE DEPTH AND LIVE IMAGE TO THE FRAME

In [ ]:
np.save( "new_K.npy",new_K)

In [ ]:
import numpy as np
np.load( "new_K.npy")